## Lab Exercise 9
### Association rule mining and co-morbidities analysis

* Please enter your answer in this Jupyter Notebook.

In [1]:
import os
import sqlalchemy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## We will use the package [`mlxtend`](http://rasbt.github.io/mlxtend/) to do association rule mining.
* To install `mlxtend` you can `conda install -c conda-forge mlxtend`

## In the next cell, please use `sqlalchemy.create_engine` to connect to MIMIC III using your own credentials
* I am reading in credentials stored in the CSV file `mimic_login_creds.csv`.
* If you want to do the same, create a similar CSV file with:
    * first row being `Username,password`
    * second row being `<your username>,<your password>`

In [3]:
creds = pd.read_csv("sample_mimic_login_creds.csv")
myUserName = str(creds.iloc[0]['Username']).strip()
myPassword = str(creds.iloc[0]['password']).strip()

server_url = "mimic-db.renci.unc.edu"
database = "mimic"

# Create Connection String
conn_str = f"{myUserName}:{myPassword}@{server_url}/{database}"

# Create Engine
engine = sqlalchemy.create_engine('postgresql://' + conn_str)


## In the next cell is a function that returns a data frame of patient_id and conditions associated with the patient from the MIMIC III EHR database.
* The SQL query used in the function is provided by Dr. Emily Pfaff (thank you, Emily !) for finding patients and their co-morbidities in the MIMIC III EHR database.
* The query is parametrized by a single value called `patient_count_threshold`, default set as 50.
* The query returns only those conditions which are present in at least `patient_count_threshold` number of patients. Increasing this threshold reduces the number of rows of data returned.
* `patient_count_threshold` is an input parameter to the function that you can change.

In [4]:
def get_comorbid_data(patient_count_threshold = 50):
    query = \
    f"with condlist as (\
    select c.concept_name, count(distinct co.person_id) as ptct \
    from omop.condition_occurrence co JOIN omop.concept c ON co.condition_concept_id = c.concept_id \
    where concept_name <> 'No matching concept' \
    group by c.concept_name \
    having count(distinct co.person_id) > {patient_count_threshold} \
    ) \
    \
    select distinct co.person_id, list.concept_name \
    from omop.condition_occurrence co JOIN omop.concept c ON co.condition_concept_id = c.concept_id \
    JOIN condlist list ON c.concept_name = list.concept_name"
    
    return pd.read_sql_query(query, engine)

### In the next cell, use the function `get_comorbid_data` to get the dataframe with the comorbidities associated with each patient.
* To keep the datasize manageable, only get the data for comorbidities that are assocaited with at least 5000 patients.
* I also suggest that you use `head` to examine the returned data frame. You should see that the dataframe has two columns, the `person_id` and `concept_name`.


In [19]:
%%time
min_patients = 500
comorbidity_data = get_comorbid_data(min_patients)
print(f"The returned date frame for a {min_patients} patient threshold has {len(comorbidity_data)} rows")
comorbidity_data.head(20)

The returned date frame for a 500 patient threshold has 391216 rows
Wall time: 11.6 s


,person_id,concept_name
0,392775850,Acquired hypothyroidism
1,392775850,Acute posthemorrhagic anemia
2,392775850,Acute renal failure syndrome
3,392775850,Acute subendocardial infarction
4,392775850,Anemia
5,392775850,Anemia due to chronic blood loss
6,392775850,Angina pectoris
7,392775850,Anticoagulant adverse reaction
8,392775850,Asthma
9,392775850,Atrial fibrillation


In [20]:
print(f"The data frame has {len(comorbidity_data['person_id'].unique())} patients")
print(f"The data frame includes {len(comorbidity_data['concept_name'].unique())} different conditions")

The data frame has 45798 patients
The data frame includes 230 different conditions


Looking for conditions that only occur in 500 patients, we can see a difference from the work we did in class.

In [21]:
new_d = comorbidity_data.groupby(['person_id'])
type(new_d)
print(new_d)

### To prepare for association rule mining, we want to convert the data frame we have into a list of lists.
* The inner list should have all the comorbidities associated with a patient as a list.
* The outer list simply holds all the inner lists.
* Consider using the pandas [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function to accomplish this. You will need extra code besides just `groupby`.
* You can check that you are on the right track by confirming that the number of inner lists is the same as the number of unique patients you found earlier.
* You can also try to print out a few of the inner lists to check.

In [22]:
new_d = comorbidity_data.groupby(['person_id'])

comorb_list = []
for person, comorb in new_d:
#     print(f"Person is {person}")
#     print(comorb['concept_name'])
    comorb_list.append(list(set(comorb['concept_name'])))
print(len(comorb_list))

45798


In [23]:
comorb_list[::500] # start, stop, step -> but why is there a 5000 step in cycling through the list?

[['Not for resuscitation',
  'Late effect of medical and surgical care complication',
  'Iatrogenic hypotension',
  'Old myocardial infarction',
  'Anemia due to chronic blood loss',
  'Coronary arteriosclerosis in native artery',
  'Hemorrhage AND/OR hematoma complicating procedure',
  'Acute renal failure syndrome',
  'Paralytic ileus',
  'Anticoagulant adverse reaction',
  'Hyperlipidemia',
  'Acute posthemorrhagic anemia',
  'Atrial fibrillation',
  'Benign hypertensive renal disease with renal failure',
  'Congestive heart failure',
  'Angina pectoris',
  'Chronic kidney disease stage 3',
  'Cerebral infarction due to embolism of cerebral arteries',
  'Low blood pressure',
  'Esophageal reflux finding',
  'Anemia',
  'Respiratory insufficiency',
  'Chronic kidney disease',
  'Asthma',
  'Acquired hypothyroidism',
  'Acute subendocardial infarction',
  'Diabetes mellitus without complication',
  'Malnutrition'],
 ['Iatrogenic hypotension',
  'Accident',
  'Coronary arteriosclerosis

### To use the `mlxtend`'s `aprori` function, you need to one-hot encode the list of conditions.
* `mlxtend` provides a method called `TransactionEncoder` that will take the list of lists and create a one-hot encoded data frame.
* Create the one-hot encoded data frame from your list of lists and look at the first 25 items using the `head` method.

In [24]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(comorb_list).transform(comorb_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head(25)

,Abdominal aortic aneurysm,Abdominal pain,Abnormal patient reaction,Accident,Accidental wound during procedure,Acidosis,Acquired hypothyroidism,Acute deep vein thrombosis of lower limb,Acute disease of cardiovascular system,Acute exacerbation of chronic obstructive bronchitis,...,Tricuspid valve disorder,Twins - both live born,Type 2 diabetes mellitus,Ulcer of foot,Upper gastrointestinal bleeding,Urinary tract infectious disease,Vascular complication of medical care,Ventilator associated pneumonia,Ventricular fibrillation,Viral hepatitis C
0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,True,True,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Now use the `mlxtend`'s `apriori` method to find the frequent itemsets in your data.
* For the 5000 patient threshold, I suggest that you try a support of 0.01.
* Remember that support is the fraction of transactions that contain an itemset.
* So a support of 0.01 means that 1% of the trasactions contain that itemset.
* With the 5000 patients threshold and a support of 0.01, you should find 404 frequent itemsets.
* The itemsets are also a dataframe, so you can use head to look at the first 25 frequent itemsets.

In [25]:
%%time
from mlxtend.frequent_patterns import apriori
support = 0.01
frequent_itemsets = apriori(df, min_support=support, use_colnames=True)
print(f"Using a support of {support}, found {len(frequent_itemsets)} frequent itemsets")
frequent_itemsets.head(25)

Using a support of 0.01, found 2114 frequent itemsets
Wall time: 2min 13s


,support,itemsets
0,0.013887,(Abdominal aortic aneurysm)
1,0.015590,(Abdominal pain)
2,0.012752,(Abnormal patient reaction)
3,0.040941,(Accident)
4,0.020045,(Accidental wound during procedure)
5,0.091227,(Acidosis)
6,0.083737,(Acquired hypothyroidism)
7,0.011332,(Acute deep vein thrombosis of lower limb)
8,0.012599,(Acute disease of cardiovascular system)
9,0.020590,(Acute exacerbation of chronic obstructive bro...


### Now add a column called `count` to your frequent itemsets dataframe that is a count of the number of items in that itemset.
* Again, use `head` to see the first 25 frequent itemsets.

In [26]:
frequent_itemsets['count'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head(25)

,support,itemsets,count
0,0.013887,(Abdominal aortic aneurysm),1
1,0.015590,(Abdominal pain),1
2,0.012752,(Abnormal patient reaction),1
3,0.040941,(Accident),1
4,0.020045,(Accidental wound during procedure),1
5,0.091227,(Acidosis),1
6,0.083737,(Acquired hypothyroidism),1
7,0.011332,(Acute deep vein thrombosis of lower limb),1
8,0.012599,(Acute disease of cardiovascular system),1
9,0.020590,(Acute exacerbation of chronic obstructive bro...,1


### Now create a new data frame of itemsets sorted by the support in descending order.
* Print out the highest and lowest support and the count associated with each.
* Again examine the first 25 rows using `head`.

In [27]:
fi = frequent_itemsets.sort_values(by = ['support'], ascending = False)
print(f"Highest support is {fi.iloc[0]['support']} with a count of {fi.iloc[0]['count']}")
print(f"Lowest support is {fi.iloc[-1]['support']} with a count of {fi.iloc[-1]['count']}")
fi.head(25)

Highest support is 0.3845801126686755 with a count of 1
Lowest support is 0.01000043670029259 with a count of 3


,support,itemsets,count
109,0.384580,(Essential hypertension),1
83,0.235272,(Coronary arteriosclerosis in native artery),1
40,0.224377,(Atrial fibrillation),1
79,0.216538,(Congestive heart failure),1
17,0.208262,(Acute renal failure syndrome),1
160,0.170772,(Newborn),1
130,0.162998,(Hyperlipidemia),1
91,0.160924,(Diabetes mellitus without complication),1
18,0.146709,(Acute respiratory failure),1
777,0.144024,"(Coronary arteriosclerosis in native artery, E...",2


###  Filter your sorted array of frequent itemsets to only show rows that have a count of at least 2.
* What are the co-occuring conditions that have the highest support?

In [29]:
fige2 = fi[fi['count'] >= 3]
fige2.head(25)

,support,itemsets,count
1827,0.108848,"(Requires vaccination, Single live birth, Newb...",3
1694,0.055352,"(Coronary arteriosclerosis in native artery, H...",3
1513,0.053059,"(Atrial fibrillation, Coronary arteriosclerosi...",3
1613,0.047862,"(Congestive heart failure, Coronary arterioscl...",3
1487,0.047862,"(Congestive heart failure, Atrial fibrillation...",3
1200,0.047382,"(Congestive heart failure, Atrial fibrillation...",3
1683,0.046116,"(Diabetes mellitus without complication, Coron...",3
1703,0.044740,"(Pure hypercholesterolemia, Coronary arteriosc...",3
1484,0.044260,"(Congestive heart failure, Atrial fibrillation...",3
1779,0.043605,"(Finding related to pregnancy, Neonatal jaundi...",3
